# Serving Deep Learning Models

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../data/wifi_location.csv')

In [ ]:
df.head()

In [ ]:
df['location'].value_counts()

In [ ]:
df.plot(figsize=(12, 8))
plt.axvline(500)
plt.axvline(1000)
plt.axvline(1500)
plt.title('Indoor location dataset')
plt.xlabel('Sample number')
plt.ylabel('Wifi strength (dB)');

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(df, hue='location');

In [ ]:
X = df.drop('location', axis=1).values
y = df['location'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.25,
                     random_state=0)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization

In [ ]:
inputs = Input(shape=X_train.shape[1:])
x = BatchNormalization()(inputs)
x = Dense(50, activation='relu')(x)
x = Dense(30, activation='relu')(x)
x = Dense(10, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
model.compile('adam',
              'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
h = model.fit(X_train, y_train,
              batch_size=128,
              epochs=40,
              verbose=0,
              validation_data=(X_test, y_test))

In [ ]:
pd.DataFrame(h.history).plot()
plt.ylim(0, 1);

In [ ]:
import os  # Miscellaneous operating system interfaces
import json  # JSON encoder and decoder
import shutil  # High-level file operations

In [ ]:
base_path = '/tmp/ztdl_models/wifi'
sub_path = 'flask'
version = 1

In [ ]:
from os.path import join

In [ ]:
export_path = join(base_path, sub_path, str(version))
export_path

In [ ]:
shutil.rmtree(export_path, ignore_errors=True)  # delete path, if exists
os.makedirs(export_path)  # create path

In [ ]:
json.loads(model.to_json())

In [ ]:
with open(join(export_path, 'model.json'), 'w') as fout:
    fout.write(model.to_json())

In [ ]:
model.save_weights(join(export_path, 'weights.h5'))

In [ ]:
os.listdir(export_path, )

In [ ]:
from tensorflow.keras.models import model_from_json

In [ ]:
with open(join(export_path, 'model.json')) as fin:
    loaded_model = model_from_json(fin.read())

In [ ]:
probas = loaded_model.predict(X_test)
probas

In [ ]:
preds = np.argmax(probas, axis=1)
preds

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, preds)

In [ ]:
loaded_model.load_weights(join(export_path, 'weights.h5'))

In [ ]:
probas = loaded_model.predict(X_test)  # class probabilities
preds = np.argmax(probas, axis=1)  # class prediction
accuracy_score(y_test, preds)  # accuracy score

## A simple deployment with Flask

In [ ]:
!cat ./model_serving/flask_serve_model.py

## Exercise 1

Open a terminal and run the script with command:

```
CUDA_VISIBLE_DEVICES="" python model_serving/flask_serve_model.py
```

Then come back here and continue

In [ ]:
import requests

In [ ]:
api_url = "http://localhost:5000/"

In [ ]:
data = X_test[:5].tolist()

In [ ]:
data

In [ ]:
payload = {'data': data}
headers = {'content-type': 'application/json'}

In [ ]:
response = requests.post(api_url,
                         data=json.dumps(payload),
                         headers=headers)

In [ ]:
response

In [ ]:
response.json()

In [ ]:
y_test[:5]

## Deployment with Tensorflow Serving

As the [documentation](https://www.tensorflow.org/serving/) says, TensorFlow Serving is a flexible, high-performance serving system for Machine Learning models, designed for production environments. TensorFlow Serving makes it easy to deploy new algorithms and experiments, while keeping the same server architecture and APIs. TensorFlow Serving provides out-of-the-box integration with TensorFlow models, but can be easily extended to serve other types of models and data.

Tensorflow Serving can accommodate both small and large deployments, and it is built for production. It is not as simple as Flask, and here we will barely scratch the surface of what it's possible with it. If you are serious about using it, we strongly recommend you take a look at the [Architecture overview](https://www.tensorflow.org/serving/architecture_overview) where many concepts like Servables, Managers and Sources are explained.

In this part of the book, we will just show you how to export a model for serving and how to ping a Tensorflow serving server. We will leave the full installation of Tensorflow serving for the end of the chapter. Installation is strongly dependent on the system you are using and is [well documented](https://www.tensorflow.org/serving/).

In [ ]:
import tensorflow as tf

In [ ]:
base_path = '/tmp/ztdl_models/wifi'
sub_path = 'tfserving'
version = 1

In [ ]:
export_path = join(base_path, sub_path, str(version))
export_path

In [ ]:
shutil.rmtree(export_path, ignore_errors=True)

In [ ]:
from tensorflow.python.saved_model.builder import SavedModelBuilder

In [ ]:
builder = SavedModelBuilder(export_path)

In [ ]:
from tensorflow.python.saved_model.signature_def_utils \
    import predict_signature_def

In [ ]:
signature = predict_signature_def(
    inputs={"inputs": model.input},
    outputs={"outputs": model.output})

In [ ]:
import keras.backend as K

In [ ]:
sess = K.get_session()

In [ ]:
builder.add_meta_graph_and_variables(
    sess=sess,
    tags=[tf.saved_model.tag_constants.SERVING],
    signature_def_map={'predict': signature})

In [ ]:
builder.save()

In [ ]:
os.listdir(export_path)

In [ ]:
os.listdir(join(export_path, 'variables'))

## Exercise 2

Go back to the terminal, stop the flask serving script and start tensorflow serving with the following command:

```
CUDA_VISIBLE_DEVICES="" tensorflow_model_server \
                           --port=8500 \
                           --model_name=wifi \
                           --model_base_path=/tmp/ztdl_models/wifi/tfserving/
```

Then come back here and continue.

In [ ]:
from grpc import insecure_channel

In [ ]:
channel = insecure_channel('localhost:8500')

In [ ]:
channel

In [ ]:
from tensorflow_serving.apis.prediction_service_pb2_grpc import PredictionServiceStub

In [ ]:
stub = PredictionServiceStub(channel)

In [ ]:
data

In [ ]:
from tensorflow.contrib.util import make_tensor_proto

In [ ]:
data_np = np.array(data)

In [ ]:
data_pb = make_tensor_proto(data_np,
                            dtype='float',
                            shape=data_np.shape)

In [ ]:
data_pb

In [ ]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

In [ ]:
request = PredictRequest()

In [ ]:
request.model_spec.name = 'wifi'

In [ ]:
request.model_spec.signature_name = 'predict'

In [ ]:
request.inputs['inputs'].CopyFrom(data_pb)

In [ ]:
request

In [ ]:
result_future = stub.Predict.future(request, 5.0)

In [ ]:
result = result_future.result()
result

In [ ]:
from tensorflow.contrib.util import make_ndarray

In [ ]:
scores = make_ndarray(result.outputs['outputs'])

In [ ]:
scores

In [ ]:
prediction = np.argmax(scores, axis=1)
prediction

In [ ]:
model.predict(np.array(data)).argmax(axis=1)